In [ ]:
install.packages("xlsx", dependency=TRUE)
library(xlsx)

install.packages("openxlsx")
library(openxlsx)

install.packages("tidyverse")
library(tidyverse)

#Packages to create well formated tables in RStuido 
install.packages("knitr")
library(knitr)
install.packages("kableExtra")
library(kableExtra)

# packages for downloading and formaing shapefiles 
install.packages("downloader")
library(downloader)

#, repos="http://cran.cnr.berkeley.edu"

install.packages("rgdal")
library(rgdal)

install.packages("RCurl")
library(RCurl)



# Background 

Improving data sharing will enable timely access to data, enhance the quality of data, and create clear channels for better management decisions.  Healthy aquatic habitat is critical to fishes, aquatic species, and water quality. Across the country long term, large scale stream habitat monitoring programs collect data for their specific objectives and within their jurisdictional boundaries.  Streams cross jurisdictional boundaries and face unprecedented pressure from changing climate, multi-use public lands, and development. To meet these stressors, we need a way to ingratiate data from multiple sources to create indicators of stream conditions across jurisdictional boundaries. 

The Pacific Northwest Aquatic Monitoring Partnership (PNAMP) with the support of the USGS faciliated a working group to describe data exchange specifications and a controlled vocabulary for sharing stream habitat metrics and metadata based on the Darwin Core standards. The full write up is published here!!!

In this Notebook we use those data exchange specifications and a cross walk from EPA National Aquatic Resources Surveys (NARS); BLM Aquatic Assessment, Inventory, and Monitoring (AIM); and USFS Aquatic and Riparian Effective Monitoring Program (AREMP) and Pacfish/Infish Biological Opinion Effectiveness Monitoring Program (PIBO) to the controlled vocabulary to build an data set. This data set intergrates metrics and site metadata from three of the four progrmas with published data sets, EPA NRSA, BLM AIM, and USFS AREMP. We also link metrics to a machine readable method documentation in MonitoringResources.org an online, publicly accessible suite of information and tools for natural resource monitoring programs, focusing on documentation of data collection and analysis methods. 

The data set it published and shared on ScienceBase and in this GitHub repository. This work is designed to serve as a piolt project, additional stream habitat data sets can be added to this by updating the field cross walk and documenting metadata in MonitoringResources.org 


## Data Exchange Specifications  

To intergrate stream habitat metrics from mutiple sources there needs to be a set of guidines and rules. Data exchange specifications are a set of guidelines and rules for using and combining information. Rigorous data exchange specifications support reuse, promote interoperability, and reduce data integration costs (Morris and Frechette 2008, Hamm 2019). The Stream Monitoring Habitat Data Exchange Specifications are a standard for exchanging metric-level habitat data based on the Darwin Core principles outlined by Wieczorek et al. in 2012. The Darwin Core standard is maintained at the GitHub repository https://github.com/tdwg/dwc. 

PNAMP facilited work with experts from each program EPA National Aquatic Resources Surveys (NARS); BLM Aquatic Assessment, Inventory, and Monitoring; and USFS Aquatic and Riparian Effective Monitoring Program (AREMP), USFS PIBO and the USGS to define the data exchange sepcifications, controlled vocabulary and finally a field cross walk from the orginal data set to the controlled vocabulary and the data exchnage specifications. The data exchange specification is manged here!!! 



###

In [ ]:
#Get and display the program data
wd <- getwd()
metadata_name <- paste0(wd, "/Data/Metadata.xlsx")
program_info <- as_tibble(read.xlsx(metadata_name, 1))
program_info 


<h3> Program Metrics </h3> 
We compiled metadata from the each of the four programs descibing the metrics and site metdata release by each program. 


In [ ]:
#Load the metric list and cross walk 
metadata <-read.xlsx(metadata_name, 3)
metadata_summary<- metadata %>%
  group_by(Category)%>%
  count(Category, sort=TRUE)

In [ ]:
s = sum(metadata_summary[,2])
paste("Across the 4 programs a total of," ,s, "metrics are calculated")

A varity of metrics are calculated within standard cataogries. 

In [ ]:
metadata_summary

We assess with list of metrics that are compiled across the three programs and the Pacific Northwest Aquatic Monitoring Parthernship .  

In [ ]:
file <- paste(wd,"/Code/Data_Organize/create_list_of_metrics.R",sep='')
source(file)
number_of_programs <- 3 #user can variabe to understand the metrics calculated across a number of programs 
metrics_list <- metrics(number_of_programs) 
count <-dim (metrics_list)
paste("There are", count[1], "metrics calculate across", number_of_programs, "programs. The table below shows the subset of metrics calculated at", number_of_programs, "or more")


In [ ]:
metrics_list

Just because the metrics are calculated across, it does not mean the data is compadiliy. Difference is field data collection methods can impact the method results. Using worok done by the BLM and other programs we outline compabiltiy hereL <a href="url">Review fild method compadbility</a> I HAVE NOT COMPLTED THAT DOCUMENT YET> 

In [ ]:
install.packages("sf")
install.packages("httr")
install.packages("data.table")
install.packages("tmap")


In [ ]:
library(sf)
 

In [ ]:
install.packages("tmap")    
library(tmap)


In [ ]:
 library(httr)

In [ ]:
 library(data.table)

# Data Sources 
## Metric Data 

Three of the four habitat programs store metric level data online, BLM AIM, EPA NARS and USFS AREMP. 

EPA NARS and USFS AREMP post data at an give interval, BLM had an ArcGIS server they update yealy. 


### ScienceBase 
To build the integrated data set we needed to document the metadata of the origin data sets in a consistent machine readable format. We chose UGSG ScienceBase to create a standard searchable machine readable documentation and that we write code against. ScienceBase is a data-sharing platform managed by the U.S. Geological Survey. It provides access to scientific data and other information using a Web Browser, and application program interface, and representational state transfer (REST)-based Web Services (Chase 2016). We write R code to download the EPA NRSA and USFS AREMP data and reformat the data to a flat data format.

### Until ScienceBase is published you will need to have an account and enter you user name and password 


In [ ]:
SBUserName  <- readline(prompt="ScienceBase User Name: ")
SBPassword  <- readline(prompt="ScienceBase Password: ")

<h3>EPA Rivers and Streams Data </h3>
EPA collects data across the Unites States, they share their metric level data here https://www.epa.gov/national-aquatic-resource-surveys/data-national-aquatic-resource-surveys. There are data sets for each two year collection period, 2004-2005 and 2008-2009, 2013-2014. The 2018-2019 data set is not yet posted we will update this text when it is releasd. 

The EPA data provided some challenges. Each data set differs slightly; the dataset has different titles, different field names for the same metrics and different formats.  For each set of data, 2004-2005 and 2008-2009 we build one containing the information we want to integrate into this analysis and the fields outlined in the data exchange schema. 

You can find the metadata and the data used to create one dataset for eacy year range on ScienceBase: 
<ul> 
<li><a href=" https://www.sciencebase.gov/catalog/item/5ea9d6a082cefae35a21ba5a"> ScienceBase Item</a></li>
<li><a href=" https://www.sciencebase.gov/catalog/item/5e3db6a4e4b0edb47be3d602"> 2008-2009 ScienceBase Item</a></li>
</ul>


In [ ]:
NRSA_metrics<- paste0(getwd(), "/Scrape EPA Data.R")
source(NRSA_metrics)
NRSA_date <- download_EPA_NRSA(SBUserName, SBPassword)



In [ ]:
#ENTER WORKFLOW DIAGRAM 

<h3>USFS AREMP Data </h3>
AREMP published thier data as a geodatabase after they release their five year report here: <a href="https://www.fs.fed.us/r6/reo/monitoring/watershed/"> https://www.fs.fed.us/r6/reo/monitoring/watershed/</a>. The data was last updated in 2015. 

You can find the metadata and the data used to create one dataset for on ScienceBase: 
<ul> 
<li><a href="https://www.sciencebase.gov/catalog/item/5e3dbb2ee4b0edb47be3d646"> AREMP ScienceBase Item</a></li>
</ul>

<h3>BLM AIM Data </h3>
BLM published thier data yearly on an ArcGIS server here: <a href="https://landscape.blm.gov/geoportal/rest/find/document;jsessionid=F4427530B569FCCAEC19400F8C19E345?searchText=isPartOf%3AAIM&contentType=liveData&start=1&max=10&f=searchpage"> https://landscape.blm.gov/geoportal/rest/find/document;jsessionid=F4427530B569FCCAEC19400F8C19E345?searchText=isPartOf%3AAIM&contentType=liveData&start=1&max=10&f=searchpage</a>. 

You can find the metadata and the data used to create one dataset for on ScienceBase: 
<ul> 
<li><a href="https://www.sciencebase.gov/catalog/item/5e3c61ffe4b0edb47be0ef27"> BLM ScienceBase Item</a></li>
</ul>

In [ ]:
 url <- list(hostname = "gis.blm.gov/arcgis/rest/services",
                                scheme = "https",
                                path = "hydrography/BLM_Natl_AIM_AquADat/MapServer/0/query",
                                query = list(
                                  where = "1=1",
                                  outFields = "*",
                                  returnGeometry = "true",
                                  f = "geojson")) %>% 
                      setattr("class", "url")
                    request <- build_url(url)
                    BLM <- st_read(request, stringsAsFactors = TRUE) #Load the file from the Data file 
                    data <- as_tibble(BLM)

## Reformating Data 
Each program releases their data in a different format and on different platforms. EPA NARS release data online in a .CSV file every five years after data collection, BLM AIM releases an ArcGIS server update every year and USFS AREMP release a ArcGIS geodatabase every five years. We design a workflow to get each program’s data from it original format into an integrated data sets (Figure 3). To build the integrated data set we document the online location and metadata of the origin data sets in ScienceBase, creating a standard searchable machine readable documentation that we write code against to pull the information. ScienceBase is an integrated data-sharing platform managed by the U.S. Geological Survey. It provides access to scientific data and other information using a Web Browser, and application program interface, and representational state transfer (REST)-based Web Services (Chase 2016). We write R code to download the EPA NRSA and USFS AREMP data and reformat the data to a flat data format. The BLM AIM data is already in a flat format, so we don’t need to do any data reformatting. Using the data dictionary, cross walk and data schema we write R code to build one data set integrating data from the three programs with accessible data sets. We publish the data set on ScienceBase and is searchable, machine readable and accessible for download of via and API so it can be used in we portals, mapping tools and programing packages (R and Python (Klein et. Al 2019).  



# Create one Data File 
To complete anaysis we create one data frame from the three  program.

In [ ]:
file <- paste0(wd,"/Code/data_organize/creating one dataframe_for loop.R")
#file <- path.expand("Code/data_organize/creating one dataframe_for loop.R")
#source("Code/data_organize/creating one dataframe_for loop.R")
source(file)
data<-one_data_frame()


# Explore In Stream Tributary Habitat Data
As a proof of concept for this analysis package, we picked 2-3 metrics (category of metrics?) as a test case to build a data dictionary, intergrade data from multiple sources and design infrastructure to serve data to interested parties.  We selected metrics that are responsive to management decisions, have low measure error as defined in the lititure (Kershner and Roper 2010, ##add other references), the method comparison (<font color="red">insert link </font>) work completed by this group. Special consideration was given to metrics identified in the <a href="https://docs.google.com/spreadsheets/d/1zeLDBvNtEaw21LR6vcnCEsOZeW5C6cNAzgTpnz8D4XI/edit#gid=596617989">speed dating exercise</a> speed dating exercise as of high importance by the three programs. 

I CAN'T Get this to run? I think it has something do do with loading packages. 



In [ ]:
install.packages('leaflet')
library(leaflet)
install.packages('dplyr')
library(dplyr)
install.packages('leaflet.extras')
library(leaflet.extras)
install.packages('DT')
library(DT)
install.packages('ggplot2')
library(ggplot2)

In [ ]:
getwd()

In [ ]:
#install.packages('shiny', dependencies = TRUE)
library(shiny)
#install.packages('tidyverse')
library(tidyverse)
#install.packages('leaflet')
library(leaflet)
#install.packages('dplyr')
library(dplyr)
#install.packages('leaflet.extras')
library(leaflet.extras)
#install.packages('DT')
library(DT)
#install.packages('ggplot2')
library(ggplot2)

In [ ]:
library(shiny)
app <- paste0(wd,"code/Visualization/app.R")
runApp(appDir=app)


In [ ]:
#Load the pallet for the map 
pal <- colorFactor(rainbow(3), data$Program)

m <- data%>% 
  leaflet() %>%
  addTiles() %>%
  addCircles(lng=~verbatimLongitude, lat= ~verbatimLatitude, color=~pal(Program), 
             popup= ~paste0("<b>",  Program, "</b>", 
                            "<br>", "<b>", "EventID ", "</b>",  eventID, "</br>",
                            "<br>", "<b>", "LocationID ", "</b>",locationID,  "</br>",
                            "<br>", "<b>", "Year ", "</b>", Year,    "</br>",
                            "<br>", "<b>", "Date ", "</b>", verbatimEventDate,    "</br>", 
                            "<br>"))%>%
                              addLegend("topright", pal=pal, values= ~Program, opacity =1)
m

In [ ]:
state_input = "OR"  
metric = "D50" # BFWDRatio' 'BFHeight' 'WetWidth' 'WetWidthToDepth' 'RPD' 'PctPool' 'Sin' 'PctDry' 'Beaver' 'StreamOrder' 'D50' 'PctFines2' 'PctFines6' 'PoolTailFines2' 'PoolTailFines6' 'LWDFreq' 'LWDVol' 'OEratio' 'MMI' 'BeaverPresent'
stream_power  = "BFWidth" #'BFWidth' 'Grad' 

data_by_state = data %>% 
      filter(State==state_input)%>%
      select(matches(metric, stream_power, "Program")
          

In [ ]:
data_by_state <-  
ggplot(data_by_state, aes(x=data_by_state[,1], y=data_by_state[,2], color=Program))+geom_point()
    plot(data_by_state)
